In [1]:
import json
import random
import os
import shutil
from urllib import urlretrieve
import sys
sys.path.append('../../')
import django,os,glob
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "dva.settings")
django.setup()

In [3]:
train_data = json.load(file("coco_sample/instances_train2014.json"))
capitons_train_data = json.load(file("coco_sample/captions_train2014.json"))
sample = random.sample(train_data['images'],150)
dirname = '/Users/aub3/coco_input/'

In [4]:
ids = set()
try:
    shutil.rmtree(dirname)
except:
    pass
os.mkdir(dirname)
for count,img in enumerate(sample):
    if (count+1)%100== 0:
        print count
    fname = os.path.join(dirname, img['file_name'])
    if not os.path.exists(fname):
        urlretrieve(img['coco_url'], fname)
    ids.add(img['id'])
from collections import defaultdict
data = defaultdict(lambda:{'image':None,'annotations':[],'captions':[]})
id_to_license = {k['id']:k for k in train_data['licenses']}
id_to_category = {k['id']:k for k in train_data['categories']}        
for entry in train_data['images']:
    if entry['id'] in ids:
        entry['license'] = id_to_license[entry['license']]
        data[entry['id']]['image'] = entry
for annotation in train_data['annotations']:
    if annotation['image_id'] in ids:
        annotation['category']=id_to_category[annotation['category_id']]
        data[annotation['image_id']]['annotations'].append(annotation)    
for annotation in capitons_train_data['annotations']:
    if annotation['image_id'] in ids:        
        data[annotation['image_id']]['captions'].append(annotation)            
with open('coco_sample_metadata.json','w') as output:
    json.dump(data,output)        

99


In [5]:
os.system("zip coco_input.zip -r {}".format(dirname))

0

In [10]:
from django.core.files.uploadedfile import SimpleUploadedFile
from dvaapp.views import handle_uploaded_file, handle_youtube_video
from dvaapp.models import Video
from dvaapp import models
from dvaapp.tasks import extract_frames, perform_face_indexing, inception_index_by_id, perform_ssd_detection_by_id, perform_yolo_detection_by_id, inception_index_ssd_detection_by_id
fname = 'coco_input.zip'
f = SimpleUploadedFile(fname, file('coco_input.zip').read(), content_type="application/zip")
v = handle_uploaded_file(f, name)
extract_frames(v.pk)
data = json.load(file('coco_sample_metadata.json'))
v.name ='mscoco_sample_500'
v.save()
video=v
models.Annotation.objects.all().filter(video=video).delete()
for frame in models.Frame.objects.all().filter(video=video):
    frame_id = str(int(frame.name.split('_')[-1].split('.')[0]))
    annotation = models.Annotation()
    annotation.video = v
    annotation.frame = frame
    annotation.full_frame = True
    annotation.metadata_text = json.dumps(data[frame_id]['image'])
    annotation.label = 'metadata'
    annotation.save()
for frame in models.Frame.objects.all().filter(video=video):
    frame_id = str(int(frame.name.split('_')[-1].split('.')[0]))
    for a in data[frame_id][u'annotations']:                                                            
        annotation = models.Annotation()
        annotation.video = v
        annotation.frame = frame
        annotation.metadata_text = json.dumps(a)
        annotation.full_frame = False        
        annotation.x = a['bbox'][0]
        annotation.y = a['bbox'][1]
        annotation.w = a['bbox'][2]
        annotation.h = a['bbox'][3]
        label,_ = models.VLabel.objects.get_or_create(video=video,label_name='coco_instance/{}/{}'.format(a[u'category'][u'supercategory'],a[u'category'][u'name']))
        annotation.label = label.label_name
        annotation.label_parent = label
        annotation.save()
    for caption in data[frame_id][u'captions']:                                                            
        annotation = models.Annotation()
        annotation.video = v
        annotation.frame = frame
        annotation.metadata_text = json.dumps(caption)
        annotation.full_frame = True        
        label,_ = models.VLabel.objects.get_or_create(video=video,label_name='coco_caption')
        annotation.label = label.label_name
        annotation.label_parent = label
        annotation.save()                

In [ ]:
inception_index_by_id(v.pk)
perform_ssd_detection_by_id(v.pk)
perform_yolo_detection_by_id(v.pk)
perform_face_indexing(v.pk)
inception_index_ssd_detection_by_id(v.pk)